In [ ]:
import os
import sys
import gensim
import pandas as pd
import re
import pickle
from nltk.tokenize import RegexpTokenizer
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from sklearn import utils
import numpy as np
from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [ ]:
clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/train.csv")
# clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_train_full.csv")
clean_train_comments.head()

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
clean_train_comments['content'] = clean_train_comments['content'].astype('str') 
clean_train_comments['tokens'] = clean_train_comments['content'].apply(tokenizer.tokenize)
clean_train_comments['sentiment'] = clean_train_comments['polarity'].astype('category').cat.codes
   
clean_train_comments.head()

In [ ]:
clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/test.csv")
# clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_testing_full.csv")
clean_test_comments.head()

In [ ]:
clean_test_comments['content'] = clean_test_comments['content'].astype('str') 
clean_test_comments["tokens"] = clean_test_comments["content"].apply(tokenizer.tokenize)
clean_test_comments['sentiment'] = clean_test_comments['polarity'].astype('category').cat.codes

clean_test_comments.head()

In [ ]:
x_train = clean_train_comments['content']
x_validation = clean_test_comments['content']
y_train = clean_train_comments['sentiment']
y_validation = clean_test_comments['sentiment']

In [ ]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
x_train = labelize_text(x_train, 'TRAIN')
x_validation = labelize_text(x_validation, 'TEST')

In [ ]:
MAX_SEQUENCE_LENGTH = 85
data_dim = 700

In [ ]:
# word2vec = Word2Vec.load("./vectorizer/tripadvisor/word2vec_300.model")
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
tfidf = pickle.load(open('./vectorizer/tripadvisor/tfidf.pickle', 'rb'))
model_dbow = Doc2Vec.load("./vectorizer/tripadvisor/model_dbow.model")
model_dmc = Doc2Vec.load("./vectorizer/tripadvisor/model_dmc.model")
model_dmm = Doc2Vec.load("./vectorizer/tripadvisor/model_dmm.model")

def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += np.append(model_dbow[word] * tfidf[word], model_dmm[word] * tfidf[word])
            count += 1
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

def build_Vector(tokens, word_size, doc_size):
    doc_vec = build_doc_Vector(tokens, doc_size)
    vec = np.zeros((MAX_SEQUENCE_LENGTH - len(tokens), doc_size + word_size))
    for word in tokens:
        try:
            word_vec = np.append(doc_vec, word2vec[word])
            vec = np.append(vec, word_vec)
        except KeyError: 
            word_vec = np.append(doc_vec, np.zeros((1, word_size)))
            vec = np.append(vec, word_vec)
            continue
    vec.reshape(MAX_SEQUENCE_LENGTH, doc_size + word_size)
    return vec

In [ ]:
train_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_train))])
val_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_validation))])

In [ ]:
batch_size = 256
num_epochs = 10
hidden_size = 10
timesteps = MAX_SEQUENCE_LENGTH
num_class = 1

In [ ]:
num_data = len(train_vecs)
num_data_val = len(val_vecs)

train_vecs = train_vecs.reshape((num_data, timesteps, data_dim))
y_train = y_train.reshape((num_data, num_class))
val_vecs = val_vecs.reshape((num_data_val, timesteps, data_dim))
y_validation = y_validation.reshape((num_data_val, num_class))

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_vecs, to_categorical(y_train), epochs=num_epochs, validation_data=(val_vecs, to_categorical(y_validation)))

In [ ]:
# model.save('./model/bi_lstm_3_pv/bi_lstm_model_01.h5')  

In [ ]:
# model = load_model('./model/bi_lstm_3_pv/bi_lstm_model_01.h5')
y_pred = model.predict(val_vecs)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_validation, y_pred, labels = [0, 1, 2], digits=8))